# Описание проекта

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.


**Цель**: 
* Спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

**Задачи**:
* Построить модель с предельно большим значением F1-меры (минимальное значение 0.59)
* Дополнительно измерить AUC-ROC, сравнивайте её значение с F1-мерой.

**Данные**: исторические данные о поведении клиентов и расторжении договоров с банком.

* RowNumber — индекс строки в данных
* CustomerId — уникальный идентификатор клиента
* Surname — фамилия
* CreditScore — кредитный рейтинг
* Geography — страна проживания
* Gender — пол
* Age — возраст
* Tenure — сколько лет человек является клиентом банка
* Balance — баланс на счёте
* NumOfProducts — количество продуктов банка, используемых клиентом
* HasCrCard — наличие кредитной карты
* IsActiveMember — активность клиента
* EstimatedSalary — предполагаемая зарплата

   **Целевой признак**
* Exited — факт ухода клиента

In [2]:
# импортируем общие библиотеки
import pandas as pd
import numpy as np
import math
import re

# импорт функций для построения моделей прогнозирования
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# импорт функции для разбивки данных на подвыборки
from sklearn.model_selection import train_test_split


# импорт функций для оценивания необходимых метрик
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# импорт функций для обработки данных
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

# Загрузка и подготовка данных

## Загрузка данных

In [474]:
try:
    data_original = pd.read_csv('Churn.csv')
    display(data_original.head())
except:
    data_original = pd.read_csv('/datasets/Churn.csv')
    display(data_original.head())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [475]:
df = data_original

**Проблема**: Названия столбцов необходимо перевести в `snake_case`.

In [476]:
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

In [477]:
df.columns = [camel_to_snake(name) for name in df.columns]
df.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

### Исследование данных

#### Проверка на пропуски и возможные дубликаты

In [478]:
print('Размер таблицы:', df.shape)
print()
print('Доля пропусков в %:')
print(df.isna().sum()/len(df)*100)

Размер таблицы: (10000, 14)

Доля пропусков в %:
row_number          0.00
customer_id         0.00
surname             0.00
credit_score        0.00
geography           0.00
gender              0.00
age                 0.00
tenure              9.09
balance             0.00
num_of_products     0.00
has_cr_card         0.00
is_active_member    0.00
estimated_salary    0.00
exited              0.00
dtype: float64


Пропуски обнаружены тольков перменной **`tenure`**.
Проверим, нет ли дубликатов в столбце **`customer_id`**. Если дубликаты есть, то можно было бы заполнить данные для наблюдений с одинаковым индетификационным номером.

In [479]:
df.duplicated(subset = ['customer_id']).sum()

0

К сожалению, естественным образом заполнить пропуски не удается. Заполнение пропусков, может привести к смещению результатов, поэтому просто избавимся от этих пропусков (9,09% от всей исходной выборки)

In [480]:
# Проверка на наличие явных дубликатов
# При наличии устраняем явные дубликаты
if df.duplicated().sum() > 0:
    df = df.drop_duplicates()
    print(f'Обнаружено и удалено {df.duplicated().sum()} штук явных дубликатов в наблюдениях.')
else:
    print('Явные дубликаты не обнаружены.')

Явные дубликаты не обнаружены.


Столбцы **`row_number`**, **`customer_id`**, **`surname`** не понадобятся для дальнейшего анализа.

In [481]:
# удаляем пропуски
df.dropna(inplace=True)

# удаляем столбцов
df = df.drop(['row_number', 'customer_id', 'surname'], axis=1)

In [482]:
df.dtypes

credit_score          int64
geography            object
gender               object
age                   int64
tenure              float64
balance             float64
num_of_products       int64
has_cr_card           int64
is_active_member      int64
estimated_salary    float64
exited                int64
dtype: object

Столбцы  **`geography`**, **`gender`** содержат текстовую информацию, поэтому возможен риск наличия неявных дубликатов характеристик.

In [483]:
df['geography'].value_counts()

France     4550
Germany    2293
Spain      2248
Name: geography, dtype: int64

In [484]:
df['gender'].value_counts()

Male      4974
Female    4117
Name: gender, dtype: int64

**Неявные дубликаты** в данных переменных **не обнаружены**.

## Подготовка данных к построению моделей

### Создание обучающей, валидационной и тестовой подвыборки

#### Преобразование категориальных переменных

In [485]:
# кодируем категориальные признаки в численные
df_ohe = pd.get_dummies(df, drop_first=True)
df_ohe.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


#### Выделение характеристик и целевого показателя

In [486]:
# сохраним в переменные наши признаки для обучения модели и целевой признак
features = df_ohe.drop(['exited'], axis=1)
target = df_ohe['exited']

In [487]:
# делим датасет на части 3:1:1
# train = 60%, valid и test по 20%
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345
)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345
)
# посмотрим на размер наших выборок
print('Train: {} {}\nValid: {} {}\nTest:  {} {}'
      .format(features_train.shape, target_train.shape, features_valid.shape,
              target_valid.shape, features_test.shape, target_test.shape))

Train: (5454, 11) (5454,)
Valid: (1818, 11) (1818,)
Test:  (1819, 11) (1819,)


### Масштабирование характеристик

In [488]:
# Определим численные характеристики
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

In [489]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

D:\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


### Выводы по загрузки и подготовки данных
* Были выявлены пропуски в столбце **`tenure`**.
* Наблюдения с пропусками были удалены.
* Явные и неявные дубликаты не были обнаружены в данных.
* Данные были разделены на при подвыборки: 
    - обучающая (60% от всех наблюдений)
    - валидационная (20% от всех наблюдений)
    - ткстовая (20% от всех наблюдений)
* Количественные характеристики были масштабированы с целью избежать ошибки, связанной с чрезмерной недооценкой и/или переоценкой влияния отдельных характеристик

# Поиск наилучшей модели

## Баланс классов

In [490]:
# посмотрим на баланс классов
target_train.value_counts(normalize=True)

0    0.793546
1    0.206454
Name: exited, dtype: float64

Присутвует дисбаланс в классах (в соотношении примерно 1:4)
Возможные пути решения проблемы с дисбалансом классов:
* Увеличение выборки за счет набоюдений, относящихся к классу 1.
* Уменьшение выборки за счет набоюдений, относящихся к классу 0.
* Взвешивание классов
* Изменение порога классификации

In [491]:
# Создаем функцию для выравнивания соотношения между классами
# путем увеличения числа наблюдений класса 1
def upsample(features, target, repeat):
    
# делим данные на признаки и целевые показатели по классам (1 или 0)
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
# формируем таблицы
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
# перемешиваем данные, чтобы повысить качество обучения    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [492]:
# Цель увеличения выборки это выровнять баланс классов
# Для этой цели достаточно увеличить класс 1 в 4 раза
features_upsampled, target_upsampled = upsample(features_train, target_train, 4) # тут оказывается нужно задать значение 5 и данные будут близко к 1:1

In [493]:
print('Баланс классов после увеличения выборки:\n{}'
      .format(target_upsampled.value_counts(normalize=True)))

Баланс классов после увеличения выборки:
1    0.509964
0    0.490036
Name: exited, dtype: float64


In [494]:
# Аналогично создаем функцию для выравнивания соотношения между классами
# путем уменьшения числа наблюдений класса 0
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [495]:
# а тут нужно задать значение 0.25 и данные будут близко к 1:1
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [496]:
print('Баланс классов после уменьшения выборки:\n{}'
      .format(target_downsampled.value_counts(normalize=True)))

Баланс классов после уменьшения выборки:
1    0.509964
0    0.490036
Name: exited, dtype: float64


In [497]:
# Создаем функцию для одновременного расчета необходимых метрик
def f1_and_auc_roc_scores(model, features, target, threshold = None):
    
    predictions = model.predict(features)
    probabilities_one_valid = model.predict_proba(features)[:, 1]
    
    if threshold == None:
        f1 = f1_score(target, predictions)
        auc_roc = roc_auc_score(target, probabilities_one_valid)
    else:
        predictions = probabilities_one_valid > threshold
        f1 = f1_score(target, predictions)
        auc_roc = roc_auc_score(target, probabilities_one_valid)
        
    return f1, auc_roc

## LogisticRegression

In [498]:
# Создаем функцию для построения модели и оценивания метрик
def logistic_regression_models(features, 
                               target, 
                               balanced = False, # Параметр, отвечающий за взвешивание классов
                               thresholds = False, # Параметр, отвечающий за взвешивание изменние порога
                               threshold_range = None, # Параметры изменения порога (диапазон)
                               threshold_step = None, # Параметры изменения порога (шаг)
                               rnd_st = 12345):
#################################################################################################    
    col_names = ['model',
                 'model_type',
                 'balanced', 
                 'threshold', 
                 'f1_score_valid', 
                 'auc_roc_valid', 
                 'f1_score_train', 
                 'auc_roc_train']
    
    data = pd.DataFrame(columns = col_names) # Создаем таблицу, куда будем сохранять результат
##################################################################################################    

# ВЫБОР МОДЕЛИ ПО ПАРАМЕТРУ ВЗВЕШИВАНИЯ КЛАССОВ    
    if balanced == False:
        model = LogisticRegression(random_state = rnd_st, 
                                   solver = 'liblinear')
        model = model.fit(features, target)
    else:
        model = LogisticRegression(random_state = rnd_st, 
                                   solver = 'liblinear', 
                                   class_weight = 'balanced')
        model = model.fit(features, target)   
        
##################################################################################################   

# РАСЧЕТ МЕТРИК С ПОМОЩЬЮ ФУНКЦИИ, НАПИСАННОЙ РАНЕЕ    
    f1_score_valid,  auc_roc_valid = f1_and_auc_roc_scores(model, 
                                                           features_valid, 
                                                           target_valid)

    f1_score_train,  auc_roc_train = f1_and_auc_roc_scores(model, 
                                                           features, 
                                                           target)      

    row = pd.DataFrame([[model,
                         'log_reg', 
                         balanced,
                         np.nan,
                         f1_score_valid,
                         auc_roc_valid,
                         f1_score_train,  
                         auc_roc_train]], columns = col_names) # Строка таблицы

    data = data.append([row], ignore_index=True) # Добавление строки в таблицу
    
###############################################################################################################
    # УЧЕТ ПОРОГА
    if thresholds == True:
        
        # Параметры для прогов классификации
        min_threshold = threshold_range[0]
        max_threshold = threshold_range[1]
        step = threshold_step
        
        for threshold in np.arange(min_threshold, max_threshold, step):
            f1_score_valid,  auc_roc_valid = f1_and_auc_roc_scores(model, 
                                                                   features_valid, 
                                                                   target_valid, 
                                                                   threshold)

            f1_score_train,  auc_roc_train = f1_and_auc_roc_scores(model, 
                                                                   features, 
                                                                   target, 
                                                                   threshold)
            row = pd.DataFrame([[model,
                                 'log_reg', 
                                 balanced,
                                 threshold,
                                 f1_score_valid,
                                 auc_roc_valid,
                                 f1_score_train,  
                                 auc_roc_train]], columns = col_names)

            data = data.append([row], ignore_index=True)        
        
    # Функция возвращает данные отсортированные по метрике F1, рассчитанной по валидационной выборке.  
    return (data.sort_values(by = 'f1_score_valid', ascending = False).reset_index(drop = True).round(4))

In [499]:
help(logistic_regression_models)

Help on function logistic_regression_models in module __main__:

logistic_regression_models(features, target, balanced=False, thresholds=False, threshold_range=None, threshold_step=None, rnd_st=12345)
    # Создаем функцию для построения модели и оценивания метрик



In [500]:
lr_models = pd.DataFrame()

In [501]:
%%time

balanced = [False, True]
thresholds = [False, True]
threshold_range = [0, 1]
threshold_step = 0.01
for balance in balanced:
    for threshold in thresholds:
        lr = logistic_regression_models(features_train, 
                                        target_train, 
                                        balance, 
                                        threshold, 
                                        threshold_range, 
                                        threshold_step)
        lr_models = lr_models.append(lr)
        lr_models = (lr_models
                     .sort_values(by = 'f1_score_valid', 
                                  ascending = False)
                     .reset_index(drop = True))

Wall time: 2.61 s


In [502]:
lr_model_no_threshold = lr_models[lr_models['threshold'].isna()].head(1)
best_lr_models = pd.concat([lr_models.head(), lr_model_no_threshold])

In [509]:
best_lr_models.drop_duplicates(inplace =True)
best_lr_models

,model,model_type,balanced,threshold,f1_score_valid,auc_roc_valid,f1_score_train,auc_roc_train
0,"LogisticRegression(class_weight='balanced', ra...",log_reg,True,0.54,0.5157,0.7778,0.5004,0.7648
1,"LogisticRegression(class_weight='balanced', ra...",log_reg,True,0.55,0.5155,0.7778,0.4995,0.7648
2,"LogisticRegression(class_weight='balanced', ra...",log_reg,True,0.52,0.5107,0.7778,0.4951,0.7648
3,"LogisticRegression(class_weight='balanced', ra...",log_reg,True,0.50,0.5097,0.7778,0.4928,0.7648
4,"LogisticRegression(class_weight='balanced', ra...",log_reg,True,NaN,0.5097,0.7778,0.4928,0.7648


**Краткий вывод**:
* Характеристики наилучшей модели среди моделей **`LogisticRegression`**:
    - F1-score = 0.5157
    - AUC-ROC = 0.7778
    - Взвешивание классов: **True**
    - Порог равен **0.54**


* Характеристики наилучшей модели среди моделей **`LogisticRegression` БЕЗ ПОПРОГА**:
    - F1-score = 0.5097
    - AUC-ROC = 0.7778
    - Взвешивание классов: **True**

## DecisionTreeClassifier

In [504]:
# Аналогичная предыдущей функция, только добавляется цикл по параметру максимальной глубины дерева
def decision_tree_classifier_models(features, 
                                    target,
                                    depth_range,
                                    balanced = False, 
                                    thresholds = False,
                                    threshold_range = None,
                                    threshold_step = None,
                                    rnd_st = 12345):
#################################################################################################
    min_depth = depth_range[0]
    max_depth = depth_range[1]
    
    col_names = ['model',
                 'model_type', 
                 'depth',
                 'balanced', 
                 'threshold', 
                 'f1_score_valid', 
                 'auc_roc_valid', 
                 'f1_score_train', 
                 'auc_roc_train']
    
    data = pd.DataFrame(columns = col_names)
##################################################################################################   
    for depth in range(min_depth, max_depth+1):
        if balanced == False:
            model = DecisionTreeClassifier(max_depth = depth,
                                           random_state = rnd_st)
            model = model.fit(features, target)
        else:
            model = DecisionTreeClassifier(max_depth = depth,
                                           random_state = rnd_st, 
                                           class_weight = 'balanced')
            model = model.fit(features, target)        
    ##################################################################################################        
        f1_score_valid,  auc_roc_valid = f1_and_auc_roc_scores(model, 
                                                               features_valid, 
                                                               target_valid)

        f1_score_train,  auc_roc_train = f1_and_auc_roc_scores(model, 
                                                               features, 
                                                               target)      

        row = pd.DataFrame([[model,
                             'dec_tree',
                             depth,
                             balanced,
                             np.nan,
                             f1_score_valid,
                             auc_roc_valid,
                             f1_score_train,  
                             auc_roc_train]], columns = col_names)
        
        data = data.append([row], ignore_index=True)
    ###############################################################################################################
        if thresholds == True:
            min_threshold = threshold_range[0]
            max_threshold = threshold_range[1]
            step = threshold_step
    
            for threshold in np.arange(min_threshold, max_threshold, step):
                f1_score_valid,  auc_roc_valid = f1_and_auc_roc_scores(model, 
                                                                       features_valid, 
                                                                       target_valid, 
                                                                       threshold)

                f1_score_train,  auc_roc_train = f1_and_auc_roc_scores(model, 
                                                                       features, 
                                                                       target, 
                                                                       threshold)
                row = pd.DataFrame([[model,
                                     'dec_tree',
                                     depth,
                                     balanced,
                                     threshold,
                                     f1_score_valid,
                                     auc_roc_valid,
                                     f1_score_train,  
                                     auc_roc_train]], columns = col_names)

                data = data.append([row], ignore_index=True)        


    return (data.sort_values(by = 'f1_score_valid', ascending = False).reset_index(drop = True).round(4))

In [505]:
dec_tree_models = pd.DataFrame()

In [507]:
%%time

balanced = [False, True]
thresholds = [False, True]
depth_range = [1, 20]
threshold_range = [0, 1]
threshold_step = 0.01
for balance in balanced:
    for threshold in thresholds:
        dec_tree = decision_tree_classifier_models(features_train, 
                                                   target_train,
                                                   depth_range,
                                                   balance, 
                                                   threshold, 
                                                   threshold_range, 
                                                   threshold_step)
        dec_tree_models = dec_tree_models.append(dec_tree)
        dec_tree_models = (dec_tree_models
                           .sort_values(by = 'f1_score_valid', 
                                        ascending = False)
                           .reset_index(drop = True))

Wall time: 48.4 s


In [510]:
dec_tree_models

,model,model_type,depth,balanced,threshold,f1_score_valid,auc_roc_valid,f1_score_train,auc_roc_train
0,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.28,0.6225,0.8346,0.6631,0.8891
1,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.29,0.6223,0.8346,0.6637,0.8891
2,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.30,0.6202,0.8346,0.6658,0.8891
3,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.33,0.6202,0.8346,0.6658,0.8891
4,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.31,0.6202,0.8346,0.6658,0.8891
...,...,...,...,...,...,...,...,...,...
4075,"DecisionTreeClassifier(max_depth=2, random_sta...",dec_tree,2,False,0.93,0.0000,0.7551,0.0000,0.7369
4076,"DecisionTreeClassifier(max_depth=2, random_sta...",dec_tree,2,False,0.94,0.0000,0.7551,0.0000,0.7369
4077,"DecisionTreeClassifier(max_depth=2, random_sta...",dec_tree,2,False,0.95,0.0000,0.7551,0.0000,0.7369
4078,"DecisionTreeClassifier(max_depth=2, random_sta...",dec_tree,2,False,0.96,0.0000,0.7551,0.0000,0.7369


In [511]:
dec_tree_model_no_threshold = dec_tree_models[dec_tree_models['threshold'].isna()].head(1)
best_dec_tree_models = pd.concat([dec_tree_models.head(), dec_tree_model_no_threshold])

In [512]:
best_dec_tree_models

,model,model_type,depth,balanced,threshold,f1_score_valid,auc_roc_valid,f1_score_train,auc_roc_train
0,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.28,0.6225,0.8346,0.6631,0.8891
1,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.29,0.6223,0.8346,0.6637,0.8891
2,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.30,0.6202,0.8346,0.6658,0.8891
3,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.33,0.6202,0.8346,0.6658,0.8891
4,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,0.31,0.6202,0.8346,0.6658,0.8891
143,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,False,NaN,0.5764,0.8346,0.6487,0.8891


**Краткий вывод**:
* Характеристики наилучшей модели среди моделей **`DecisionTreeClassifier`**:
    - **F1-score = 0.6225** (достаточный результат)
    - AUC-ROC = 0.8346
    - Максимальная глубина дерева 7
    - Взвешивание классов: **False**
    - Порог равен **0.28**


* Характеристики наилучшей модели среди моделей **`DecisionTreeClassifier` БЕЗ ПОПРОГА**:
    - F1-score = 0.5764
    - AUC-ROC = 0.8346
    - Максимальная глубина дерева 7
    - Взвешивание классов: **False**

## RandomForestClassifier

In [513]:
# Аналогичная функция, только добавляется еще цикл для числа деревьев
def random_forest_classifier_models(features, 
                                    target,
                                    depth_range,
                                    est_range,
                                    balanced = False, 
                                    thresholds = False, 
                                    threshold_range = None,
                                    threshold_step = None,
                                    rnd_st = 12345):
#################################################################################################
    min_depth = depth_range[0]
    max_depth = depth_range[1]
    
    min_est = est_range[0]
    max_est = est_range[1]
    
    min_threshold = threshold_range[0]
    max_threshold = threshold_range[1]
    step = threshold_step
    
    col_names = ['model',
                 'model_type',
                 'depth',
                 'n_estimators',
                 'balanced', 
                 'threshold', 
                 'f1_score_valid', 
                 'auc_roc_valid', 
                 'f1_score_train', 
                 'auc_roc_train']
    
    data = pd.DataFrame(columns = col_names)
##################################################################################################
    for depth in range(min_depth, max_depth+1):
        for n_est in range(min_est, max_est+1):
            if balanced == False:
                model = RandomForestClassifier(max_depth = depth,
                                               n_estimators = n_est,
                                               random_state = rnd_st)
                model = model.fit(features, target)
            else:
                model = RandomForestClassifier(max_depth = depth,
                                               n_estimators = n_est,
                                               random_state = rnd_st, 
                                               class_weight = 'balanced')
                model = model.fit(features, target)        
        ##################################################################################################        
            f1_score_valid,  auc_roc_valid = f1_and_auc_roc_scores(model, 
                                                                   features_valid, 
                                                                   target_valid)

            f1_score_train,  auc_roc_train = f1_and_auc_roc_scores(model, 
                                                                   features, 
                                                                   target)      

            row = pd.DataFrame([[model,
                                 'rand_forest',
                                 depth,
                                 n_est,
                                 balanced,
                                 np.nan,
                                 f1_score_valid,
                                 auc_roc_valid,
                                 f1_score_train,  
                                 auc_roc_train]], columns = col_names)

            data = data.append([row], ignore_index=True)
        ###############################################################################################################
            if thresholds == True:  
                min_threshold = threshold_range[0]
                max_threshold = threshold_range[1]
                step = threshold_step
   
                for threshold in np.arange(min_threshold, max_threshold, step):
                    f1_score_valid,  auc_roc_valid = f1_and_auc_roc_scores(model, 
                                                                           features_valid, 
                                                                           target_valid, 
                                                                           threshold)

                    f1_score_train,  auc_roc_train = f1_and_auc_roc_scores(model, 
                                                                           features, 
                                                                           target, 
                                                                           threshold)
                    row = pd.DataFrame([[model,
                                         'rand_forest',
                                         depth,
                                         n_est,
                                         balanced,
                                         threshold,
                                         f1_score_valid,
                                         auc_roc_valid,
                                         f1_score_train,  
                                         auc_roc_train]], columns = col_names)

                    data = data.append([row], ignore_index=True)        


    return (data.sort_values(by = 'f1_score_valid', ascending = False).reset_index(drop = True).round(4))

In [514]:
rand_forest_models = pd.DataFrame()

**ВНИМАНИЕ!!!** Код ниже будет выполняться примерно **6 мин**.
Если это долго, то поменяйте параметры перед циклом. Но тогда выводы могут измениться.

In [516]:
%%time

balanced = [False, True]
thresholds = [False, True]
depth_range = [4, 10]
est_range = [90, 100]
threshold_range = [0, 1]
threshold_step = 0.1
for balance in balanced:
    for threshold in thresholds:
        rand_forest = random_forest_classifier_models(features_train, 
                                                      target_train,
                                                      depth_range,
                                                      est_range,
                                                      balance, 
                                                      threshold, 
                                                      threshold_range, 
                                                      threshold_step)
        
        rand_forest_models = rand_forest_models.append(rand_forest)
        rand_forest_models = (rand_forest_models
                              .sort_values(by = 'f1_score_valid', 
                                           ascending = False)
                              .reset_index(drop = True))

Wall time: 6min 14s


In [517]:
rand_forest_model_no_threshold = rand_forest_models[rand_forest_models['threshold'].isna()].head(1)
best_rand_forest_models = pd.concat([rand_forest_models.head(), rand_forest_model_no_threshold])

In [518]:
best_rand_forest_models

,model,model_type,depth,n_estimators,balanced,threshold,f1_score_valid,auc_roc_valid,f1_score_train,auc_roc_train
0,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,95,False,0.3,0.6568,0.8678,0.6721,0.8997
1,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,94,False,0.3,0.6550,0.8678,0.6718,0.8998
2,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,99,False,0.3,0.6541,0.8682,0.6718,0.8997
3,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,92,False,0.3,0.6541,0.8677,0.6733,0.8997
4,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,93,False,0.3,0.6532,0.8678,0.6727,0.8999
23,"(DecisionTreeClassifier(max_depth=8, max_featu...",rand_forest,8,96,True,NaN,0.6469,0.8696,0.7166,0.9310


**Краткий вывод**:

Выбор наилучшей модели среди моделей **`RandomForestClassifier`** затруднен в силу того, что полный перебор может занять слишком много времени (но это возможно сделать с моим кодом).

Выбранные ограничения для определения наилучшей модели:

* Максимальная глубина деревьев от 4 до 10 (выбор бы обусловлен результатами оценивания моделей дерева решения)


* Общее число деревьев от 90 до 100 (лес так лес, зачем нам три сосны).


* Характеристики наилучшей модели с учетом отмеченных ограничений среди моделей **`RandomForestClassifier`**:
    - **F1-score = 0.6568** (достаточный результат)
    - AUC-ROC = 0.8678
    - Максимальная глубина дерева = 7
    - Число деревьев = 95
    - Взвешивание классов: **False**
    - Порог равен **0.3**


* Характеристики наилучшей модели с учетом отмеченных ограничений среди моделей **`RandomForestClassifier` БЕЗ ПОПРОГА**:
    - **F1-score = 0.6469** (достаточный результат)
    - AUC-ROC = 0.8696
    - Максимальная глубина дерева = 8
    - Число деревьев = 96
    - Взвешивание классов: **True**


In [519]:
best_models = pd.concat([best_rand_forest_models, best_dec_tree_models, best_lr_models])

In [520]:
best_models = best_models.sort_values(by = 'f1_score_valid', ascending = False).reset_index(drop=True)
best_models

,model,model_type,depth,n_estimators,balanced,threshold,f1_score_valid,auc_roc_valid,f1_score_train,auc_roc_train
0,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,95,False,0.30,0.6568,0.8678,0.6721,0.8997
1,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,94,False,0.30,0.6550,0.8678,0.6718,0.8998
2,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,99,False,0.30,0.6541,0.8682,0.6718,0.8997
3,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,92,False,0.30,0.6541,0.8677,0.6733,0.8997
4,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,93,False,0.30,0.6532,0.8678,0.6727,0.8999
5,"(DecisionTreeClassifier(max_depth=8, max_featu...",rand_forest,8,96,True,NaN,0.6469,0.8696,0.7166,0.9310
6,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.28,0.6225,0.8346,0.6631,0.8891
7,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.29,0.6223,0.8346,0.6637,0.8891
8,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.31,0.6202,0.8346,0.6658,0.8891
9,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.30,0.6202,0.8346,0.6658,0.8891


In [521]:
def test_f1_and_auc_roc_scores(model, features, target, threshold):
      
    predictions = model.predict(features)
    probabilities_one_valid = model.predict_proba(features)[:, 1]
    
    if math.isnan(threshold):
        f1 = f1_score(target, predictions)
        auc_roc = roc_auc_score(target, probabilities_one_valid)
    else:
        predictions = probabilities_one_valid > threshold
        f1 = f1_score(target, predictions)
        auc_roc = roc_auc_score(target, probabilities_one_valid)
        
    return f1, auc_roc

In [522]:
col_names = ['model', 
             'model_type', 
             'depth', 
             'n_estimators', 
             'balanced', 
             'threshold',
             'f1_score_test', 
             'auc_roc_test']

test_best_models = pd.DataFrame(columns = col_names)

for i in range(len(best_models)):
    f1_test, auc_roc_test = test_f1_and_auc_roc_scores(best_models.loc[i, 'model'], 
                                                       features_test, 
                                                       target_test, 
                                                       best_models.loc[i, 'threshold'])
    
    test_best_models.loc[i, best_models.columns[0:6]] = best_models.loc[i, best_models.columns[0:6]]
    
    test_best_models.loc[i, 'f1_score_test'] = f1_test
    test_best_models.loc[i, 'auc_roc_test'] = auc_roc_test



In [523]:
test_best_models = test_best_models.sort_values(by = 'f1_score_test', ascending = False)
test_best_models

,model,model_type,depth,n_estimators,balanced,threshold,f1_score_test,auc_roc_test
2,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,99,False,0.3,0.626471,0.863543
1,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,94,False,0.3,0.618768,0.863661
4,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,93,False,0.3,0.618076,0.86352
0,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,95,False,0.3,0.617862,0.863518
3,"(DecisionTreeClassifier(max_depth=7, max_featu...",rand_forest,7,92,False,0.3,0.617862,0.863227
5,"(DecisionTreeClassifier(max_depth=8, max_featu...",rand_forest,8,96,True,NaN,0.593789,0.861327
7,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.29,0.585294,0.828017
6,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.28,0.583942,0.828017
10,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.33,0.578711,0.828017
8,"DecisionTreeClassifier(max_depth=7, random_sta...",dec_tree,7,NaN,False,0.31,0.578711,0.828017


# ОБЩИЙ ВЫВОД


* Характеристики наилучшей модели с учетом отмеченных ограничений среди моделей **`RandomForestClassifier`** по показателю F1-score **на тестовой выборке**:
    - **F1-score = 0.6265** на тестовой выборке (достаточный результат)
    - AUC-ROC = 0.8635
    - Максимальная глубина дерева = 7
    - Число деревьев = 99
    - Взвешивание классов: **False**
    - Порог равен **0.3**


* Лучшая модель по показателю F1-score **на валидационной выборке** показала 4-ый результат **на тестовой выборке**. Её характеристики:
    - **F1-score = 0.6179** на тестовой выборке (достаточный результат)
    - AUC-ROC = 0.8635
    - Максимальная глубина дерева = 7
    - Число деревьев = 95
    - Взвешивание классов: **False**
    - Порог равен **0.3**


* Характеристики наилучшей модели с учетом отмеченных ограничений среди моделей **БЕЗ ПОПРОГА**:
    - **F1-score = 0.6469** (достаточный результат)
    - AUC-ROC = 0.8696
    - Максимальная глубина дерева = 8
    - Число деревьев = 96
    - Взвешивание классов: **True**
